In [ ]:
#This Code snippet creates a dataset of Appfolio tenants matched with home purchases
import csv
with open ('ArlingtonHomePurchases.csv', 'r') as purchases, open ('TenantsMetroDCWithMoveOut.csv', 'r') as tenants:
    purchasesReader = csv.reader(purchases, delimiter=',')
    tenantsReader = csv.reader(tenants, delimiter=',')
    
    tenantFirstNameIndex = 1
    tenantLastNameIndex = 2
    tenantIDIndex = 0
    tenantMoveOutIndex = 3
    
    purchaseDateIndex = 11
    purchaseNameIndex = 9
 
    
    
    tenantSetList = []
    for tenantRow in tenantsReader:
            tenantNameSet = set([tenantRow[tenantFirstNameIndex].lower(), tenantRow[tenantLastNameIndex].lower()])
            tenantID = tenantRow[tenantIDIndex]
            tenantMoveOut = tenantRow[tenantMoveOutIndex]
            tenantSetList.append([tenantID, tenantMoveOut, tenantNameSet])
            
            
    for purchaseRow in purchasesReader:
        purchaseNameSet = set(purchaseRow[purchaseNameIndex].lower().split(' '))
        for tenantNameSet in tenantSetList:
            intersection = purchaseNameSet.intersection(tenantNameSet[2])
            if len(intersection) >= 2:
                print(tenantNameSet[0],',', purchaseRow[0],',', tenantNameSet[1], ',', purchaseRow[purchaseDateIndex][0:10], ',', ' '.join(tenantNameSet[2]), ',', purchaseRow[purchaseNameIndex].lower())
            
                

In [ ]:
#This code snippet counts the number of unique tenants matched as home buyers
import csv
with open ('TenantHomeBuyers.csv') as tenantBuyers:
    tenantBuyersReader = csv.reader(tenantBuyers, delimiter=",")
    tenantNameIndex = 1
    tenantNameSet = set([])
    
    for row in tenantBuyersReader:
        tenantNameSet.add(row[tenantNameIndex])
    print(len(tenantNameSet))

In [ ]:
#This snippet further filters the tenant-home buyer matches to include tenants who moved out of their apartment within
#six months of purchasing a home and removing common names who appear multiple times in the same year in the home buyers
# data set
import csv
import datetime

with open ('TenantHomeBuyerswithDates.csv') as tenantBuyers, open ('ArlingtonHomePurchases.csv') as purchases:
    tenantBuyersReader = csv.reader(tenantBuyers, delimiter=",")
    purchasesReader = csv.reader(purchases, delimiter=',')
    tenantNameIndex = 1
    purchaseDateIndex = 3
    moveOutDateIndex = 2
    purchaseNameIndex = 5
    tenantBuyers = []
    
    purchaseDateIndexOrg = 11
    purchaseNameIndexOrg = 9
    nameDateDict = {}
    
    def purchaseNameDateDictCreator():
        firstLine = True
        for purchaseRow in purchasesReader:
            if firstLine:
                firstLine = False
                continue
            purchaseDate = purchaseRow[purchaseDateIndexOrg][0:10]
            purchaseName = purchaseRow[purchaseNameIndexOrg]
            if purchaseName in nameDateDict:
                nameDateDict[purchaseName] += [purchaseDate]
            else: 
                nameDateDict[purchaseName] = [purchaseDate]
        return nameDateDict
    
    def namesAppearingOften(freq, nameDateDict):
        tempDateArray = []
        names = []
        for key in nameDateDict.keys():
            tempDateArray = []
            for dateString in nameDateDict[key]:
                tempDateArray += [datetime.datetime.strptime(dateString, '%Y-%m-%d')]
            prev = datetime.datetime.strptime('01-01-50', '%m-%d-%y')
            nameFlag = False
            for date in sorted(tempDateArray):
                if abs((date - prev).days) < freq:
                    nameFlag = True
                prev = date
            if nameFlag: names += [key.lower()]
        return names
    
    frequentNames = namesAppearingOften(365, purchaseNameDateDictCreator())
    print('Frequent Names:' + str(len(frequentNames)))
    
    for row in tenantBuyersReader:
        moveOutString = row[moveOutDateIndex].strip()
        if moveOutString:
            moveOutDate = datetime.datetime.strptime(moveOutString, '%Y-%m-%d')
            purchaseDate = datetime.datetime.strptime(row[purchaseDateIndex].strip(), '%Y-%m-%d')
            date_difference = abs((moveOutDate - purchaseDate).days)
            date_window = 180
            purchaseName = row[purchaseNameIndex]
            if date_difference < date_window  and (purchaseName not in frequentNames):
                tenantBuyers.append(row)
    print(len(tenantBuyers))
    
    tenantSet = set()
    for tenant in tenantBuyers:
        tenantSet.add(tenant[0])
    print(len(tenantSet))
    
    print(tenantBuyers)
    